In [ ]:
## KSE and K-means clustering

In [99]:
import numpy as np
import pandas as pd
from scipy import stats

In [116]:
data = pd.read_csv('~/Desktop/netflow_sample.rtf', sep = ',', skiprows =8)
data.columns = ["srcAddr","dstAddr","srcPort","dstPort","prot","bytes","pkts","dur","rate","firstTime","lastTime","sas","das","smk","dmk","maxrate"]
clean_data = data[["bytes","pkts","dur","rate"]]

,bytes,pkts,dur,rate
0,80,2,0.000,0.00
1,256,2,0.000,0.00
2,3394,52,0.212,128075.47
3,120,2,0.000,0.00
4,184,4,9.405,156.51


In [113]:
## function adopted from paper, KSE
def euc_dis(p1,p2):
    raw = 0
    for i in range(0,3):
        raw += (p1[i] - p2[i])**2
    return np.sqrt(raw)
def KSE_Test(df, fraction):
    # initiate two sample dfs, one distance matrix, and one kse score.
    # generate 2 random samples.
    sample1 = df.sample(frac = fraction, replace = False)
    sample2 = df.sample(frac = fraction, replace = False)
    kse_score = [None] * len(df) # empty list to store KSE scores
#   distance_matrix = np.matrix([[None] * int(len(df) * fraction) , 
#                               [None] * int(len(df) * fraction)])
    distance_matrix = [[0 for x in range(len(sample1))] 
                       for y in range(len(sample2))] 
    # calculate distance from all points in sample1 to all points in sample2
    for i in range(0,len(sample1)):
        for j in range(0,len(sample2)):
            distance_matrix[i][j] = euc_dis(sample1.iloc[i,],sample2.iloc[j,])
    
    # calculate average KS-score for each instance in df.
    for i in range(0, len(df)):
        observed_instance = df.iloc[i,]
        distance_array = [None] * len(sample2)
        for j in range(0,len(sample2)):
            distance_array[j] = euc_dis(observed_instance,sample2.iloc[j,])
        sum = 0
        for k in range(0, len(sample1)):
            ks_score = stats.ks_2samp(distance_array, distance_matrix[k])[0]
            sum += ks_score
        average = sum/len(sample1)
        kse_score[i] = average
    
    return kse_score 

In [119]:
test = KSE_Test(clean_data, 0.8)
sum(test)

212.56492400881868

In [129]:
#### read in the real thing
data1 = pd.read_csv('~/Desktop/netFlow-02-11-2017.csv', sep = ',')
#data2 = pd.read_csv('~/Desktop/netFlow-02-18-2017.csv', sep = ',')

In [138]:
# see what the tails look like
data1.head()
#data1.tail()
#data2.head()
#data2.tail()

,Unnamed: 0,id,srcAddr,dstAddr,srcPort,dstPort,prot,bytes,pkts,dur,rate,firstTime,lastTime,maxrate
0,0,0,75.102.143.245,128.143.2.7,53,47759,17,168,2,0.000,0.00,2017-02-11 17:35:39.736,2017-02-11 17:35:39.736,0.0
1,1,1,128.143.2.7,104.156.84.32,44985,53,17,160,2,0.000,0.00,2017-02-11 17:38:35.023,2017-02-11 17:38:35.023,0.0
2,2,2,114.33.89.12,128.143.7.14,22929,23,6,80,2,0.000,0.00,2017-02-11 17:18:03.556,2017-02-11 17:18:03.556,0.0
3,3,3,23.253.131.243,128.143.3.7,53,44564,17,256,2,0.000,0.00,2017-02-11 17:06:05.767,2017-02-11 17:06:05.767,0.0
4,4,4,174.218.129.232,128.143.8.199,5040,80,6,3394,52,0.212,128075.47,2017-02-11 17:10:45.977,2017-02-11 17:10:46.189,128075.0


In [ ]:
# the two datasets have drastically different length
# choose only data1, break the dataset into each hours

In [145]:
# clean_data1 = data1[["bytes","pkts","dur","rate"]]
# filter bidirectional traffic with regards to collab.
data_collab = data1[(((data1.srcAddr == '128.143.2.75') & (data1.srcPort == 443)) | ((data1.dstAddr =='128.143.2.75') & (data1.dstPort == 443)))]

In [146]:
len(data_collab) / len(data1)

0.042764526388952055

In [147]:
data_collab

,Unnamed: 0,id,srcAddr,dstAddr,srcPort,dstPort,prot,bytes,pkts,dur,rate,firstTime,lastTime,maxrate
150,150,150,184.2.66.47,128.143.2.75,55868,443,6,6042,22,0.292,165534.25,2017-02-11 17:03:40.502,2017-02-11 17:03:40.794,165534.0
157,157,157,73.152.165.101,128.143.2.75,50481,443,6,13398,246,0.482,222373.39,2017-02-11 17:33:46.094,2017-02-11 17:33:46.957,0.0
191,191,191,128.143.2.75,73.147.28.3,443,58145,6,1500,14,0.087,137931.20,2017-02-11 17:22:15.676,2017-02-11 17:22:15.799,128367.0
235,235,235,76.104.25.80,128.143.2.75,52631,443,6,5538,22,0.189,234412.54,2017-02-11 17:31:20.722,2017-02-11 17:31:20.959,225608.0
249,249,249,128.143.2.75,108.18.207.19,443,50914,6,7116,20,0.375,151808.00,2017-02-11 17:20:12.260,2017-02-11 17:20:12.674,150101.0
286,286,286,128.143.2.75,73.152.164.153,443,52576,6,1500,14,0.100,119999.83,2017-02-11 17:32:08.784,2017-02-11 17:32:08.921,111680.0
328,328,328,128.143.2.75,76.104.31.115,443,64422,6,1332,14,0.120,88800.08,2017-02-11 17:07:12.604,2017-02-11 17:07:12.949,83466.0
350,350,350,98.249.42.36,128.143.2.75,36997,443,6,3162,14,0.228,110947.37,2017-02-11 17:28:19.765,2017-02-11 17:28:19.993,110947.0
352,352,352,128.143.2.75,109.9.170.171,443,55351,6,1540,12,0.412,29902.92,2017-02-11 17:29:24.417,2017-02-11 17:29:24.945,28349.0
374,374,374,128.143.2.75,96.255.6.219,443,54653,6,1638,14,0.075,174719.89,2017-02-11 17:23:18.371,2017-02-11 17:23:18.680,0.0


In [159]:
data_collab1 = data1[(((data1.srcAddr == '128.143.2.75') ) | ((data1.dstAddr =='128.143.2.75') & (data1.dstPort == 443)))]

In [158]:
#data_collab.bytes.value_counts()

In [160]:
data_collab2 = data1[(((data1.srcAddr == '128.143.2.75') ) | ((data1.dstAddr =='128.143.2.75')))]

In [169]:
data_collab1[data_collab1.srcAddr == '128.143.2.75'].srcPort.value_counts()

443    579642
80        297
Name: srcPort, dtype: int64

In [170]:
data_collab[data_collab.srcAddr == '128.143.2.75'].srcPort.value_counts()

443    579642
Name: srcPort, dtype: int64

In [171]:
data_collab2[data_collab2.srcAddr == '128.143.2.75'].srcPort.value_counts()

443    579642
80        297
Name: srcPort, dtype: int64

In [172]:
data1[data1.srcAddr == '128.143.2.75'].srcPort.value_counts()

443    579642
80        297
Name: srcPort, dtype: int64

In [174]:
len(data1[data1.dstAddr == '128.143.2.75'].dstPort.value_counts())

84

In [1]:
len(data_collab)

NameError: name 'data_collab' is not defined

In [177]:
clean_collab = data_collab[["bytes","pkts","dur","rate"]]

In [ ]:
### next step..
# applying our approach on this dataset

# 1. Clean up this dataset
KSE_result = KSE_Test(clean_collab, 0.8)